In [80]:
import torch
import torch.nn.functional as F
import numpy as np
from scipy.linalg import logm


def kl_single_pair(p, q):
    a = torch.log(torch.linalg.det(q[1])/torch.linalg.det(p[1]))
    print('a:', a)
    b = torch.trace(torch.linalg.inv(q[1]) * p[1])
    print('b:', b)
    c = torch.sum((p[0] - q[0]) * torch.linalg.inv(q[1]) * (p[0] - q[0]))
    print('c:', c)
    n = p[1].shape[0]
    return 0.5 * (a - n + b + c)


p = (mu1, Sigma1) = torch.tensor([0.2, 0.1, 0.5, 0.4]), torch.diag(torch.tensor([0.14, 0.52, 0.2, 0.4]))
q = (mu2, Sigma2) = torch.tensor([0.3, 0.6, -0.5, -0.8]), torch.diag(torch.tensor([0.24, 0.02, 0.31, 0.51]))
print('p|q')
print(kl_single_pair(p, q))  # 20.28295597572157
print('q|p')
print(kl_single_pair(q, p))  # 5.883921991346153
print('p|p')
print(kl_single_pair(p, p))  # 0.0


p|q
a: tensor(-2.0379)
b: tensor(28.0128)
c: tensor(18.5910)
tensor(20.2830)
q|p
a: tensor(2.0379)
b: tensor(4.5777)
c: tensor(9.1522)
tensor(5.8839)
p|p
a: tensor(0.)
b: tensor(4.)
c: tensor(0.)
tensor(0.)


In [83]:
def multivar_continue_KL_divergence(mu:torch.Tensor, sigma:torch.Tensor):
    n_tokens, embed_dim = mu.shape[0], mu.shape[1]
    mu1_mu2_diff_mat =  mu.unsqueeze(1) -  mu.unsqueeze(0).repeat(n_tokens, 1, 1)  # [n_tokens, n_tokens, embed_dim]
    sigma = sigma.unsqueeze(1) * torch.eye(embed_dim)  # [n_tokens, embed_dim ,embed_dim]
    sigma_log_det = torch.linalg.det(sigma)
    sigma_log_det_ratio = torch.log((1 / sigma_log_det).unsqueeze(1) * sigma_log_det)
    sigma_inverse = torch.linalg.inv(sigma)  # [n_tokens, embed_dim ,embed_dim]
    sigma_mut_trace = torch.sum(sigma.unsqueeze(1) * sigma_inverse, dim=(-1,-2))
    return 0.5 * (sigma_log_det_ratio - embed_dim + sigma_mut_trace + torch.sum(mu1_mu2_diff_mat.unsqueeze(-1) * sigma_inverse * mu1_mu2_diff_mat.unsqueeze(-1), dim=(-1,-2)))
    # a = torch.log(torch.linalg.det(q[1])/torch.linalg.det(p[1]))
    # print('a:', a)
    # b = torch.trace(torch.linalg.inv(q[1]) * p[1])
    # print('b:', b)
    # c = torch.sum((p[0] - q[0]) * torch.linalg.inv(q[1]) * (p[0] - q[0]))
    # print('c:', c)
    # n = p[1].shape[0]
    # print(n)
    # return 0.5 * (a - n + b + c)


# p = (mu1, Sigma1) = torch.tensor([0.2, 0.1, 0.5, 0.4]), torch.diag(torch.tensor([0.14, 0.52, 0.2, 0.4]))
# q = (mu2, Sigma2) = torch.tensor([0.3, 0.6, -0.5, -0.8]), torch.diag(torch.tensor([0.24, 0.02, 0.31, 0.51]))

mu = torch.tensor(
    [
        [0.2, 0.1, 0.5, 0.4],
        [0.3, 0.6, -0.5, -0.8]
    ]
)

sigma = torch.tensor(
    [
        [0.14, 0.52, 0.2, 0.4],
        [0.24, 0.02, 0.31, 0.51]
    ]
)
print(mu.shape)

print(multivar_continue_KL_divergence(mu, sigma))  # 20.28295597572157
# print(multivar_continue_KL_divergence(q, p))  # 5.883921991346153
# print(multivar_continue_KL_divergence(p, p))  # 0.0
torch.set_printoptions(profile='full')

torch.Size([2, 4])
tensor([[ 0.0000, 20.2830],
        [ 5.8839,  0.0000]])
